In [ ]:
%load_ext kamu
import kamu
con = kamu.connect()

In [ ]:
%%sql
select * from "com.github.kamu-data.stargazers" limit 3

In [ ]:
%%sql -o gh_stars -q
select
    starred_at as event_time,
    row_number() over (order by starred_at) as stars
from "com.github.kamu-data.stargazers"

In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
from datetime import datetime
pd.set_option('max_colwidth', None)

In [ ]:
# Using current numbers and lerping them over time using github history
# Valid as of: 2023-03-02
medium = 28 + 32
youtube = 65
discord = 40
mailchimp = 106
linkedin = 82

In [ ]:
def lerp(a: float, b: float, t: float) -> float:
    return (1 - t) * a + t * b

def adlerp(start, end, distribution, offset):
    d = []
    m = distribution.max()
    for x in distribution.values:
        t = float(x) / float(m)
        if t < offset:
            toff = 0.0
        else:
            toff = (t - offset) / (1 - offset)
        v = lerp(float(start), float(end), toff)
        d.append(int(v))
    return d


community = gh_stars.copy()

community["Medium"] = adlerp(0, medium, gh_stars.stars, 0.0)
community["YouTube"] = adlerp(0, youtube, gh_stars.stars, 0.3)
community["Discord"] = adlerp(0, discord, gh_stars.stars, 0.6)
community["News"] = adlerp(0, mailchimp, gh_stars.stars, 0.0)
community["LinkedIn"] = adlerp(0, linkedin, gh_stars.stars, 0.5)

community = community.rename(columns={"stars": "GitHub"})
community

In [ ]:
from bokeh.models.formatters import DatetimeTickFormatter

formatter = DatetimeTickFormatter(months='%b %Y')

hv.renderer('bokeh').theme = None
community.hvplot.area(
    x="event_time", 
    y=["GitHub", "Medium", "News", "LinkedIn", "YouTube", "Discord"],
    color=["#74A5FF", "#FFC382", "#A45A7F", "#E14ADF", "#FF6F41", "#2ABC9B"],
    #title="Community",
    xlabel="Date",
    ylabel="Audience",
    legend="top_left",
    line_color=None,
    xformatter=DatetimeTickFormatter(months='%b %Y'),
    #grid=True,
    height=500,
    width=800,
)